# Reproduction of Adult dataset experiments

In this notebook we reproduce the results from Table 2 of the DECAF paper. We compare various methods for generating debiased data using the DECAF model against synthetic data generated using benchmark models GAN, WGAN-GP and FairGAN. As described in the paper we run all experiments (as implemented in this notebook) 10 times and avarage the results.

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from data import load_adult, preprocess_adult
from metrics import DP, FTU
from train import train_decaf, train_fairgan, train_vanilla_gan, train_wgan_gp


## Loading data

In [ ]:
dataset = load_adult()
dataset.head()

Preprocess the data next in order to make it suitable for training models on.

In [ ]:
dataset = preprocess_adult(dataset)
dataset.head()

Split the dataset into train and test folds. Test fold size is 2000.

In [ ]:
# Split data into train and testing sets
dataset_train, dataset_test = train_test_split(dataset, test_size=2000,
                                               stratify=dataset['income'])

print('Size of train set:', len(dataset_train))
print('Size of test set:', len(dataset_test))

### Defining the DAG

We need to define a DAG which captures the biases of the dataset. As described in the DECAF paper normally a causal discovery algorithm is used. In this notebook we simply copy the DAG which as described in the Zhang et al. paper which is the one also used in the DECAF paper.

In [ ]:
# Define DAG for Adult dataset
dag = [
    # Edges from race
    ['race', 'occupation'],
    ['race', 'income'],
    ['race', 'hours-per-week'],
    ['race', 'education'],
    ['race', 'marital-status'],

    # Edges from age
    ['age', 'occupation'],
    ['age', 'hours-per-week'],
    ['age', 'income'],
    ['age', 'workclass'],
    ['age', 'marital-status'],
    ['age', 'education'],
    ['age', 'relationship'],
    
    # Edges from sex
    ['sex', 'occupation'],
    ['sex', 'marital-status'],
    ['sex', 'income'],
    ['sex', 'workclass'],
    ['sex', 'education'],
    ['sex', 'relationship'],
    
    # Edges from native country
    ['native-country', 'marital-status'],
    ['native-country', 'hours-per-week'],
    ['native-country', 'education'],
    ['native-country', 'workclass'],
    ['native-country', 'income'],
    ['native-country', 'relationship'],
    
    # Edges from marital status
    ['marital-status', 'occupation'],
    ['marital-status', 'hours-per-week'],
    ['marital-status', 'income'],
    ['marital-status', 'workclass'],
    ['marital-status', 'relationship'],
    ['marital-status', 'education'],
    
    # Edges from education
    ['education', 'occupation'],
    ['education', 'hours-per-week'],
    ['education', 'income'],
    ['education', 'workclass'],
    ['education', 'relationship'],
    
    # All remaining edges
    ['occupation', 'income'],
    ['hours-per-week', 'income'],
    ['workclass', 'income'],
    ['relationship', 'income'],
]

def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(dataset, dag)
print(dag_seed)

It's also necessary to define edges we want to remove from the DAG in order to meet the various fairness criteria described in the paper.

In [ ]:
def create_bias_dict(df, edge_map):
    """
    Convert the given edge tuples to a bias dict used for generating
    debiased synthetic data.
    """
    bias_dict = {}
    for key, val in edge_map.items():
        bias_dict[df.columns.get_loc(key)] = [df.columns.get_loc(f) for f in val]
    
    return bias_dict

# Bias dictionary to satisfy FTU
bias_dict_ftu = create_bias_dict(dataset, {'income': ['sex']})
print('Bias dict FTU:', bias_dict_ftu)

# Bias dictionary to satisfy DP
bias_dict_dp = create_bias_dict(dataset, {'income': [
    'occupation', 'hours-per-week', 'marital-status', 'education', 'sex',
    'workclass', 'relationship']})
print('Bias dict DP:', bias_dict_dp)

# Bias dictionary to satisfy CF
bias_dict_cf = create_bias_dict(dataset, {'income': [
    'marital-status', 'sex']})
print('Bias dict CF:', bias_dict_cf)

## Experiments

We have loaded and preprocessed the data and we are ready to run the experiments. For each experiment we train a generative model, sample synthetic data from the trained model and then obtain metrics by training and evaluating a downstream multi-layer perceptron using the test fold we generated in the previous section. We use the MLP model from `sklearn` with default parameters which matches the settings described in Appendix D of the paper.

In [ ]:
X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
base_clf = MLPClassifier()
base_clf.fit(X_train, y_train)

def eval_model(dataset_train, dataset_test, use_base_clf=True):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=['income']), dataset_train['income']
    X_test, y_test = dataset_test.drop(columns=['income']), dataset_test['income']

    if use_base_clf:
        y_train = base_clf.predict(X_train)

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

### Original dataset

As a benchmark we want to first train the downstream model on the original dataset.

In [ ]:
eval_model(dataset_train, dataset_test, use_base_clf=False)

In the following sections we train various models in order to reproduce the results from Table 2 of the DECAF paper.

### GAN

In [ ]:
synth_data = train_vanilla_gan(dataset_train)
synth_data.head()

In [ ]:
eval_model(synth_data, dataset_test)

### WGAN-GP

In [ ]:
synth_data = train_wgan_gp(dataset_train)
synth_data.head()

In [ ]:
eval_model(synth_data, dataset_test)

### FairGAN

In [ ]:
synth_data = train_fairgan(dataset_train)
synth_data.head()

In [ ]:
eval_model(synth_data, dataset_test, use_base_clf=False)

### DECAF

#### DECAF-ND

In [ ]:
synth_data = train_decaf(dataset_train, dag_seed)
synth_data.head()

In [ ]:
eval_model(synth_data, dataset_test)

#### DECAF-FTU

In [ ]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_ftu)
synth_data.head()

In [ ]:
eval_model(synth_data, dataset_test)

#### DECAF-CF

In [ ]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_cf)
synth_data.head()

In [ ]:
eval_model(synth_data, dataset_test)

#### DECAF-DP

In [ ]:
synth_data = train_decaf(dataset_train, dag_seed, biased_edges=bias_dict_dp)
synth_data.head()

In [ ]:
eval_model(synth_data, dataset_test)